<a href="https://colab.research.google.com/github/Sounakray2003/Asmadiya-tech/blob/main/BERTClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q kaggle pandas scikit-learn transformers datasets torch

In [ ]:
import os
import zipfile

# Upload kaggle.json (you can upload it manually if in notebook)
kaggle_json_path = "kaggle.json"  # make sure this file exists
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

# Download the dataset
!kaggle datasets download -d uciml/sms-spam-collection-dataset

# Unzip
with zipfile.ZipFile("sms-spam-collection-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("data")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#  Load data with correct delimiter and encoding
df = pd.read_csv("./data/spam.csv", encoding='ISO-8859-1')

#  The original dataset has extra columns — keep only the necessary ones


print(df.shape)
print(df.columns)
print(df['v1'].value_counts())
df.isnull().sum()

# Infer text & label columns
text_col = df.select_dtypes(include='object').columns[1]
label_col = [col for col in df.columns if df[col].nunique() < 50 and col != text_col][0]

# Drop nulls and encode labels
df = df[[text_col, label_col]].dropna()
df.columns = ['text', 'label']  # standardize column names
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split: Train / Val / Test
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)


print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_texts(df):
    return tokenizer(list(df['text']), padding=True, truncation=True, return_tensors="pt")

train_encodings = tokenize_texts(train_df)
val_encodings = tokenize_texts(val_df)
test_encodings = tokenize_texts(test_df)


In [7]:
import torch

class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = list(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

train_dataset = SpamDataset(train_encodings, train_df['label'])
val_dataset = SpamDataset(val_encodings, val_df['label'])
test_dataset = SpamDataset(test_encodings, test_df['label'])


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
num_labels = df['label'].nunique()
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",     # <- validate every epoch
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # <- this is your validation set
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
eval_result = trainer.evaluate()
print(eval_result)


In [ ]:
trainer.eval_dataset = test_dataset
test_metrics = trainer.evaluate()
print("Test set metrics:", test_metrics)


In [ ]:
def predict_message(text: str):
    model.eval()

    # Ensure model and inputs are on the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    label_map = {0: "ham", 1: "spam"}
    return label_map[prediction]

# Example usage
while True:
    msg = input("Enter a message (or type 'exit' to quit): ")
    if msg.lower() == "exit":
        break
    result = predict_message(msg)
    print(f"Prediction: {result}")


